In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import random
# tải về dữ liệu tiếng Việt
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# đường dẫn tới file chứa dữ liệu câu hỏi và câu trả lời
data_path = './chatDataSet.txt'

# đọc dữ liệu từ file
with open(data_path, 'r', encoding='utf8') as f:
    lines = f.read().split('\n')
    
# xử lý dữ liệu
lemmatizer = WordNetLemmatizer()

In [3]:
# tách câu thành từng từ và đưa về dạng nguyên mẫu
def preprocess_text(text):
    text = text.lower().strip()
    words = nltk.word_tokenize(text)
    return [lemmatizer.lemmatize(w) for w in words]

# tạo từ điển chứa các từ trong dữ liệu
word_dict = {}
for line in lines:
    words = preprocess_text(line)
    for word in words:
        if word not in word_dict:
            word_dict[word] = len(word_dict)

In [4]:
# tạo câu hỏi và câu trả lời dưới dạng chuỗi số
question_seqs = []
answer_seqs = []

for line in lines:
    if line:
        line = line.strip()
        parts = line.split(":")
        if len(parts) == 2:
            question = preprocess_text(parts[0])
            answer = preprocess_text(parts[1])
            question_seqs.append([word_dict[q] for q in question])
            answer_seqs.append([word_dict[a] for a in answer])

In [5]:
# chuyển đổi định dạng chuỗi số sang ma trận
maxlen = 50
question_data = np.zeros((len(question_seqs), maxlen), dtype=np.int32)
answer_data = np.zeros((len(answer_seqs), maxlen), dtype=np.int32)

for i, seq in enumerate(question_seqs):
    if seq:
        question_data[i, -len(seq):] = np.array(seq)[:maxlen]

for i, seq in enumerate(answer_seqs):
    if seq:
        answer_data[i, -len(seq):] = np.array(seq)[:maxlen]

In [6]:
question_data.shape

(12859, 50)

In [7]:
answer_data.shape

(12859, 50)

In [10]:
vocab_size = len(word_dict)
embedding_dim = 128
lstm_units = 256
embedding_dim = 50
from keras.layers import TimeDistributed, Bidirectional

def create_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen),
        LSTM(lstm_units, return_sequences=True),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [11]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            250000    
                                                                 
 lstm_1 (LSTM)               (None, 50, 256)           314368    
                                                                 
 time_distributed_1 (TimeDis  (None, 50, 5000)         1285000   
 tributed)                                                       
                                                                 
Total params: 1,849,368
Trainable params: 1,849,368
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
# huấn luyện mô hình
batch_size = 64
epochs = 50
with tf.device('/gpu:0'):
    history = model.fit(question_data, tf.keras.utils.to_categorical(answer_data, num_classes=vocab_size),
                        batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/50
161/161 [==============================] - 143s 877ms/step - loss: 1.8401 - accuracy: 0.8975 - val_loss: 1.1318 - val_accuracy: 0.9018
Epoch 2/50
161/161 [==============================] - 139s 860ms/step - loss: 1.0236 - accuracy: 0.9032 - val_loss: 0.9755 - val_accuracy: 0.9018
Epoch 3/50
161/161 [==============================] - 139s 866ms/step - loss: 0.8844 - accuracy: 0.9032 - val_loss: 0.8714 - val_accuracy: 0.9018
Epoch 4/50
161/161 [==============================] - 140s 872ms/step - loss: 0.8334 - accuracy: 0.9032 - val_loss: 0.8638 - val_accuracy: 0.9018
Epoch 5/50
161/161 [==============================] - 139s 865ms/step - loss: 0.8226 - accuracy: 0.9032 - val_loss: 0.8486 - val_accuracy: 0.9021
Epoch 6/50
161/161 [==============================] - 136s 847ms/step - loss: 0.8150 - accuracy: 0.9032 - val_loss: 0.8365 - val_accuracy: 0.9021
Epoch 7/50
161/161 [==============================] - 142s 880ms/step - loss: 0.8102 - accuracy: 0.9032 - val_loss: 0.8410 -

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_loss'], c = 'coral', label = 'validation loss line')
plt.plot(history.history['loss'], c = 'blue', label = 'train loss line')
legend = plt.legend(loc='upper center')
plt.show()

In [9]:
# save model
model.save(r'./botChatv1.h5')

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def predict_answer(question):
    # tiền xử lý câu hỏi
    question = preprocess_text(question)
    seq = [word_dict[word] for word in question]
    seq = pad_sequences([seq], maxlen=maxlen)
    print(question)
    # dự đoán câu trả lời
    prediction = model.predict(seq)
    prediction = prediction[0][-len(question):]
    prediction = np.argmax(prediction, axis=1)

    # chuyển đổi lại sang dạng văn bản
    print(prediction)
    rev_word_dict = {v: k for k, v in word_dict.items()}
    answer = [rev_word_dict[idx] for idx in prediction]
    answer = ' '.join(answer)

    return answer

In [12]:
question = 'hi'
print(predict_answer(question))

['hi']
1/1 [==============================] - 1s 693ms/step
[0]
ngon
